In [ ]:
include("../load.jl")


In [ ]:
# BBF 1
m = Model(with_optimizer(CPLEX_SILENT))
@variable(m, -1 <= x <= 1)
@variable(m, -1 <= y <= 1)
gm = GlobalModel(model = m)
add_nonlinear_constraint(gm, :((x,y) -> x^2*(0.2-y) + y))
bbf = gm.bbfs[1]
bbf.constraint

In [ ]:
# BBF 2 
gm = minlp(true)
bbf = gm.bbfs[1]
bbf.constraint

In [ ]:
# BBF 3 
gm = minlp(true)
bbf = gm.bbfs[2]
bbf.constraint

In [ ]:
# BBF 1 as an objective
m = Model(with_optimizer(CPLEX_SILENT))
@variable(m, -1 <= x <= 1)
@variable(m, -1 <= y <= 1)
@variable(m, -1 <= obj <=1)
gm = GlobalModel(model = m)
add_nonlinear_constraint(gm, :((x,y, obj) -> obj - (x^2*(0.2-y) + y)))
bbf = gm.bbfs[1]
bbf.constraint

In [ ]:
using Plots
""" Plots the feasible and infeasible samples of BBF in two axes. """
function plot_2d(bbf::Union{BlackBoxFunction, DataConstraint}; axis1 = 1, axis2 = 2)
    feas_idxs = findall(x -> x .>= 0, bbf.Y)
    infeas_idxs = findall(x -> x .< 0, bbf.Y)
    scatter(Matrix(bbf.X)[feas_idxs,axis1], Matrix(bbf.X)[feas_idxs, axis2],
            legend = false, color = :green,  fmt = :png)
    scatter!(Matrix(bbf.X)[infeas_idxs, axis1], Matrix(bbf.X)[infeas_idxs, axis2],
            title = "Feasible vs. infeasible samples",
    legend = false, color = :red,  fmt = :png)
end
""" Plots whether or not model predictions are correct, in 2D. """
function plot_2d_predictions(bbf::Union{BlackBoxFunction, DataConstraint}; axis1 = 1, axis2 = 2)
    Y = IAI.predict(bbf.learners[end], bbf.X)
    trues = [i for i=1:length(Y) if Y[i] == (bbf.Y[i] >= 0)];
    falses = [i for i=1:length(Y) if Y[i] != (bbf.Y[i] >= 0)];
    scatter(Matrix(bbf.X)[trues, axis1], Matrix(bbf.X)[trues, axis2], legend = false,
            color = :purple,  fmt = :png)
    scatter!(Matrix(bbf.X)[falses, axis1], Matrix(bbf.X)[falses, axis2], legend = false,
             title = "Correct vs. incorrect predictions",
             color = :yellow,  fmt = :png)
end

In [ ]:
# Uniform sampling
sample_and_eval!(bbf, lh_iterations=10)
plot_2d(bbf)

In [ ]:
# KNN sampling
sample_and_eval!(bbf)
plot_2d(bbf)

In [ ]:
# Vanilla training
learn_constraint!(bbf)

In [ ]:
# Plotting predictions
plot_2d_predictions(bbf)

In [ ]:
bbf.accuracies

In [ ]:
show_trees(bbf)